<a href="https://colab.research.google.com/github/neryguilherme/thegamelibrary/blob/main/classificacao_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

<a href="https://colab.research.google.com/github/neryguilherme/thegamelibrary/blob/main/Classifica%C3%A7%C3%A3o_Mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# Carregar os dados
banco = pd.read_parquet('/content/games_preprocessed.parquet')

banco

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Indie, Casual, Outros","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Indie, Action","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Indie, Action, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Indie, Casual, Outros","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Indie, Outros","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0 - 0,0,0,0.00,0,"Innovative Gameplay SUR5 brings a brand new, f...","['English', 'Polish', 'Spanish - Latin America']",...,0,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Indie, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71712,2125160,Prison Life 2,"Mar 30, 2023",0 - 20000,6,0,10.79,0,Core Features Real players only! All Guards an...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Indie, Casual, Outros","First-Person,Detective,Social Deduction,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",0 - 20000,0,0,5.99,0,Architecture Zeitgeist is a game created with ...,"['English', 'Japanese', 'Portuguese - Brazil',...",...,0,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual, Outros","Casual,Simulation,Puzzle,Exploration,Immersive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",0 - 0,0,0,2.39,0,"Development progress: prologue map (finished),...",['Simplified Chinese'],...,0,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Indie, Action, Casual, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [27]:
# prompt:  para cada linha na coluna "Genres" verifique se os valores "Indie", "Action", "Casual" estão presentes, caso existam mantenha eles na lista. Em seguida verifique se há outros valores que NÃO sejam "Indie", "Action", "Casual", caso existam substitua todos por "Outros". Em seguida ordene os valores da célula de acordo com a ordem seguinte: ["Indie","Action", "Casual", "Outros"]

def process_genres(genres_str):
    genres = []
    if isinstance(genres_str, str):
      genres = genres_str.split(',')

    processed_genres = []
    others_present = False

    for genre in genres:
        genre = genre.strip()
        if genre in ["Indie", "Action", "Casual"]:
            processed_genres.append(genre)
        elif genre != "":
            others_present = True

    if others_present:
        processed_genres.append("Outros")

    ordered_genres = []
    for target_genre in ["Indie", "Action", "Casual", "Outros"]:
      if target_genre in processed_genres:
        ordered_genres.append(target_genre)

    return ", ".join(ordered_genres)


banco['Genres'] = banco['Genres'].apply(process_genres)
banco['Genres']

,Genres
0,"Indie, Casual, Outros"
1,"Indie, Action"
2,"Indie, Action, Outros"
3,"Indie, Casual, Outros"
4,"Indie, Outros"
...,...
71711,"Indie, Outros"
71712,"Indie, Casual, Outros"
71713,"Casual, Outros"
71714,"Indie, Action, Casual, Outros"


In [44]:
# Preenchimento de valores faltantes
for col in banco.columns:
    if banco[col].dtype == 'object':
        banco[col] = banco[col].fillna(banco[col].mode()[0])  # Preenche com a moda
    elif pd.api.types.is_numeric_dtype(banco[col]):
        banco[col] = banco[col].fillna(banco[col].median())  # Preenche com a mediana
banco


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Indie, Casual, Outros","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Indie, Action","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Indie, Action, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Indie, Casual, Outros","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Indie, Outros","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0 - 0,0,0,0.00,0,"Innovative Gameplay SUR5 brings a brand new, f...","['English', 'Polish', 'Spanish - Latin America']",...,0,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Indie, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71712,2125160,Prison Life 2,"Mar 30, 2023",0 - 20000,6,0,10.79,0,Core Features Real players only! All Guards an...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Indie, Casual, Outros","First-Person,Detective,Social Deduction,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",0 - 20000,0,0,5.99,0,Architecture Zeitgeist is a game created with ...,"['English', 'Japanese', 'Portuguese - Brazil',...",...,0,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual, Outros","Casual,Simulation,Puzzle,Exploration,Immersive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",0 - 0,0,0,2.39,0,"Development progress: prologue map (finished),...",['Simplified Chinese'],...,0,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Indie, Action, Casual, Outros","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [45]:
# prompt: aplique o label encoder nos atributos

# Aplicar Label Encoding aos atributos categóricos
label_encoders = {}
x_column = banco.copy()
for column in x_column.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    x_column[column] = le.fit_transform(x_column[column])
    label_encoders[column] = le
x_column

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,23057,3607,1,0,0,19.99,0,23392,9273,...,0,0,0,26237,22614,4293,13,31908,34772,50347
1,655370,62118,3488,1,0,0,0.99,0,49461,3664,...,0,0,0,29606,32773,4574,8,29681,57021,11773
2,1732930,30303,3198,1,0,0,4.99,0,32483,7444,...,0,0,0,5860,5041,924,11,30826,26032,46442
3,1355720,26378,1874,1,0,0,5.99,0,25331,3516,...,0,0,0,24890,21455,976,13,124,13049,39829
4,1139950,62950,1294,1,0,0,0.00,0,5634,9098,...,0,0,0,36483,31424,4437,14,30395,5419,28277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,49224,14,0,0,0,0.00,0,30669,7281,...,0,0,0,18484,15872,3508,14,30826,38113,63073
71712,2125160,44325,2664,1,6,0,10.79,0,14619,2512,...,0,0,0,778,723,2458,13,25759,37196,65885
71713,2318860,3794,2620,1,0,0,5.99,0,8482,6814,...,0,0,0,21735,25501,924,6,20303,41817,66064
71714,2180330,23825,134,0,0,0,2.39,0,16805,9583,...,0,0,0,42313,36565,924,10,30826,38663,62976


In [46]:
X = x_column.drop(columns=['Genres'])
X.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [47]:
# Escalonar os dados numéricos
scaler = StandardScaler()
numerical_cols = X.select_dtypes(include=['number']).columns
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])
X

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Tags,Screenshots,Movies
0,-1.970885,-0.609276,1.248325,-0.447085,-0.024282,-0.145418,1.153087,-0.04121,-0.550053,0.568330,...,-0.09687,-0.057575,-0.065114,-0.056889,0.451959,0.474674,0.820258,0.432102,-0.042294,0.935097
1,-0.909117,1.292152,1.149448,-0.447085,-0.024282,-0.145418,-0.562958,-0.04121,0.780267,-1.664751,...,-0.09687,-0.057575,-0.065114,-0.056889,0.725161,1.419140,0.976020,0.266444,1.048669,-1.141058
2,0.892161,-0.256552,0.908487,-0.447085,-0.024282,-0.145418,-0.201685,-0.04121,-0.086133,-0.159840,...,-0.09687,-0.057575,-0.065114,-0.056889,-1.200474,-1.159060,-1.047220,0.351616,-0.470853,0.724920
3,0.261606,-0.447615,-0.191622,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-0.451105,-1.723674,...,-0.09687,-0.057575,-0.065114,-0.056889,0.342726,0.366923,-1.018395,-1.932185,-1.107465,0.368991
4,-0.099080,1.332653,-0.673543,-0.447085,-0.024282,-0.145418,-0.652373,-0.04121,-1.456257,0.498658,...,-0.09687,-0.057575,-0.065114,-0.056889,1.282838,1.293725,0.900079,0.319556,-1.481597,-0.252769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,1.604592,0.664492,-1.737093,-0.803532,-0.024282,-0.145418,-0.652373,-0.04121,-0.178703,-0.224734,...,-0.09687,-0.057575,-0.065114,-0.056889,-0.176756,-0.152119,0.385123,0.351616,0.121529,1.620044
71712,1.547823,0.426017,0.464788,-0.447085,-0.023247,-0.145418,0.322160,-0.04121,-0.997746,-2.123391,...,-0.09687,-0.057575,-0.065114,-0.056889,-1.612589,-1.560498,-0.196905,-0.025298,0.076565,1.771393
71713,1.871617,-1.546969,0.428228,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-1.310922,-0.410659,...,-0.09687,-0.057575,-0.065114,-0.056889,0.086878,0.743074,-1.047220,-0.431148,0.303152,1.781028
71714,1.640047,-0.571891,-1.637385,-0.803532,-0.024282,-0.145418,-0.436513,-0.04121,-0.886193,0.691749,...,-0.09687,-0.057575,-0.065114,-0.056889,1.755610,1.771676,-1.047220,0.351616,0.148498,1.614823


In [48]:
#y = le.fit_transform(name_column)
k = x_column['Genres'] #Definindo k para a coluna Genres
k = le.fit_transform(k)
k

array([13,  8, 11, ...,  6, 10, 12])

In [49]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, k, test_size=0.2)

In [56]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ... (Your existing code for data loading and preprocessing) ...

# Prepare data for RNN
# Assuming 'Genres' is already label encoded (k)
max_words = 1000  # Adjust based on vocabulary size
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(x_column['Genres'].astype(str))
sequences = tokenizer.texts_to_sequences(x_column['Genres'].astype(str))
padded_sequences = pad_sequences(sequences, padding='post', truncating='post', maxlen=5)


X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(padded_sequences, k, test_size=0.2, random_state=42)


# RNN Model
model = Sequential()
model.add(Embedding(max_words, 8, input_length=5)) # adjust embedding dimension
model.add(LSTM(64))
model.add(Dense(len(np.unique(k)), activation='softmax')) # output layer with softmax
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Training
model.fit(X_train_rnn, y_train_rnn, epochs=10, validation_data=(X_test_rnn, y_test_rnn))


# Evaluate the Model
y_pred_rnn_prob = model.predict(X_test_rnn)
y_pred_rnn = np.argmax(y_pred_rnn_prob, axis=1)
print(classification_report(y_test_rnn, y_pred_rnn, zero_division=0.0))


accuracy_rnn = accuracy_score(y_test_rnn, y_pred_rnn)
precision_rnn = precision_score(y_test_rnn, y_pred_rnn, average='weighted', zero_division=0.0)
recall_rnn = recall_score(y_test_rnn, y_pred_rnn, average='weighted', zero_division=0.0)
f1_rnn = f1_score(y_test_rnn, y_pred_rnn, average='weighted', zero_division=0.0)

print(f"RNN Accuracy: {accuracy_rnn}")
print(f"RNN Precision: {precision_rnn}")
print(f"RNN Recall: {recall_rnn}")
print(f"RNN F1-score: {f1_rnn}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.7394 - loss: 0.8077 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 2/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 4.8138e-04
Epoch 3/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 1.0000 - loss: 3.7404e-04 - val_accuracy: 1.0000 - val_loss: 1.4975e-04
Epoch 4/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 1.0000 - loss: 1.2160e-04 - val_accuracy: 1.0000 - val_loss: 5.4718e-05
Epoch 5/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 1.0000 - loss: 4.5140e-05 - val_accuracy: 1.0000 - val_loss: 2.1129e-05
Epoch 6/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 1.0000 - loss: 1.7367e-05 - val_accuracy: 1.0000 - val_loss: 8.0707e-06
Epoch 7/10
1793/1793 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 1.0000 - loss: 6.6620e-06 - val_accuracy: 1.0000 - val_loss: 3.2135e-06
Epoch 8/10
1793/1793 ━━━━━━━━━━━━━━━━━━